# Introduction to Pyresample

https://pyresample.readthedocs.io/en/latest/

This package seems a bit more speciallized, and does not have as tight of integration with xarray like xESMF and Verde do. If working with satellite or swath data, this is not one to miss!

#### Integration with xarray

This is on the to-do list: https://pyresample.readthedocs.io/en/latest/roadmap.html?highlight=xarray#xarray-and-geoxarray

### Prerequisites

Knowing your way around pandas, xarray, numpy is beneficial. This is not deisgned to be an introduction to any of those packages. Would do this notebook after doing the xESMF one!

# Imports

In [61]:
import pandas as pd
import numpy as np

from appdirs import *

import dask.array as da

import pyresample
from pyresample import image, geometry
from pyresample.bilinear import NumpyBilinearResampler

from xarray import DataArray
from pyresample.bilinear import XArrayBilinearResampler

import os


import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

In [47]:
%load_ext watermark
%watermark --iversions

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
dask      : 2022.7.0
matplotlib: 3.5.2
sys       : 3.9.13 | packaged by conda-forge | (main, May 27 2022, 17:01:00) 
[Clang 13.0.1 ]
numpy     : 1.21.5
pandas    : 1.4.3
pyresample: 1.25.1



# Resampling of gridded data using pyresample

Link to this turtorial is here: https://pyresample.readthedocs.io/en/latest/swath.html#pyresample-bilinear

We will be deconstructing it a bit to get into the details, but all of the code is from the above link.

In [48]:
target_def = geometry.AreaDefinition('areaD',
                                     'Europe (3km, HRV, VTC)',
                                     'areaD',
                                     {'a': '6378144.0', 'b': '6356759.0',
                                      'lat_0': '50.00', 'lat_ts': '50.00',
                                      'lon_0': '8.00', 'proj': 'stere'},
                                     800, 800,
                                     [-1370912.72, -909968.64,
                                      1029087.28, 1490031.36])

Unlike using xESMF, this does not depend or work with xarray:

In [49]:
print('target def type', type(target_def))

target def type <class 'pyresample.geometry.AreaDefinition'>


In [50]:
data = DataArray(da.from_array(np.fromfunction(lambda y, x: y*x, (500, 100))), dims=('y', 'x'))
type(data)

xarray.core.dataarray.DataArray

In [51]:
lons = da.from_array(np.fromfunction(lambda y, x: 3 + x * 0.1, (500, 100)))
lats = da.from_array(np.fromfunction(lambda y, x: 75 - y * 0.1, (500, 100)))
source_def = geometry.SwathDefinition(lons=lons, lats=lats)

In [52]:
resampler = XArrayBilinearResampler(source_def, target_def, 30e3)
result = resampler.resample(data)

/Users/tmartin/opt/anaconda3/envs/ml_scratch/lib/python3.9/site-packages/pyproj/crs/crs.py:1282: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)


In [53]:
result.to_dataset()

<xarray.Dataset>
Dimensions:                                   (x: 800, y: 800)
Coordinates:
  * x                                         (x) float64 -1.369e+06 ... 1.02...
  * y                                         (y) float64 1.489e+06 ... -9.08...
Data variables:
    reshape-9bdf528de498c9e7d5b7070d8e65ae41  (y, x) float64 dask.array<chunksize=(800, 800), meta=np.ndarray>

In [54]:
data.to_dataset()

<xarray.Dataset>
Dimensions:                                 (y: 500, x: 100)
Dimensions without coordinates: y, x
Data variables:
    array-c6304ed7e79d5c61e4fb47cdb25246bf  (y, x) float64 dask.array<chunksize=(500, 100), meta=np.ndarray>

In [62]:
from tempfile import gettempdir
cache_file = os.path.join(gettempdir(), "bilinear_resampling_luts.zarr")


In [63]:
resampler.save_resampling_info(cache_file)

new_resampler = XArrayBilinearResampler(source_def, target_def, 30e3)
new_resampler.load_resampling_info(cache_file)
result = new_resampler.resample(data)

ContainsGroupError: path '' contains a group

### Summary

Verde and xESMF are two different gridding packages, with two different aims. They both work well with xarray!